In [1]:
import numpy as np
import scipy.interpolate as interpolate
import scipy.fftpack as fft
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot
output_notebook()

<div class="bk-root">
        <a href="http://bokeh.pydata.org" target="_blank" class="bk-logo bk-logo-small bk-logo-notebook"></a>
        <span id="04300f5b-d444-4550-a49e-035d8a2b618f">Loading BokehJS ...</span>
    </div>

<div class="bk-root">
        <a href="http://bokeh.pydata.org" target="_blank" class="bk-logo bk-logo-small bk-logo-notebook"></a>
        <span id="04300f5b-d444-4550-a49e-035d8a2b618f">Loading BokehJS ...</span>
    </div>

### Problem 1

In [2]:
d = np.arange(30, 76, 5)
i = [0.85, 0.67, 0.52, 0.42, 0.34, 0.28, 0.24, 0.21, 0.18, 0.15]

In [3]:
def polynomial_fit(x, y, deg=None):
    if deg is None:
        deg = len(x) - 1
    p = np.polyfit(x, y, deg)
    return np.poly1d(p)

quad = polynomial_fit(d, i, 2)

In [4]:
def exp(x, y):
    log = interpolate.interp1d(x, np.log2(y), "linear")
    return np.vectorize(lambda val: 2**log(val))

exp = exp(d, i)

In [5]:
poly = polynomial_fit(d, i)

In [6]:
spline = interpolate.CubicSpline(d, i)

In [7]:
x = np.linspace(30, 75, 1000)
ys = [interpolator(x) for interpolator in (quad, exp, poly, spline)]

In [8]:
def make_figures(x, ys, fig_title, titles, colors):
    fig = figure(width=400, plot_height=400, title=fig_title)
    for y, title, color in zip(ys, titles, colors):
        fig.line(x, y, legend=title, line_color=color)
    return fig

In [9]:
figs = make_figures(x, ys, "Interpolating functions", 
                    ["quadratic", "exponential", "polynomial", "cubic spline"], 
                    ["red", "blue", "green", "orange"])

In [10]:
show(figs)

### Problem 2

In [11]:
def trig_interp(func, n=17):
    x_interp = np.arange(0, n+1)*(2*np.pi/n)
    coeffs = fft.fft(func(x_interp))/n
    def interpolator(x):
        y = np.zeros(len(x))
        for k in range(1, int((n+1)/2)):
            y = y + coeffs[k]*np.exp(1j*(k-1)*x)
        for k in range(int((n+1)/2+1), n):
            y = y + coeffs[k]*np.exp(1j*(-n+k-1)*x)
        return np.real(y)
    return interpolator